In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

!pip install pysam


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 44.3 MB/s eta 0:00:00


In [4]:
# prompt: load a sam file


import pysam

samfile = pysam.AlignmentFile("/content/drive/My Drive/Colab Notebooks/test.sam", "r")

header = samfile.header
print(header)

for read in samfile:
    print(read)
    break



# Remember to close the file when you're done
#samfile.close()

@SQ	SN:CHROMOSOME_I	LN:15072423
@SQ	SN:CHROMOSOME_II	LN:15279345
@SQ	SN:CHROMOSOME_III	LN:13783700
@SQ	SN:CHROMOSOME_IV	LN:17493793
@SQ	SN:CHROMOSOME_V	LN:20924149

I	20	*	2	1	27M1D73M	*	0	0	CCTAGCCCTAACCCTAACCCTAACCCTAGCCTAAGCCTAAGCCTAAGCCTAAGCCTAAGCCTAAGCCTAAGCCTAAGCCTAAGCCTAAGCCTAAGCCTAA	array('B', [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 31, 33, 30, 23, 33, 30, 33, 32, 31, 31, 35, 35, 33, 34, 35, 35, 34, 33, 34, 31, 34, 35, 34, 35, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34])	[('XG', 1), ('XM', 5), ('XN', 0), ('XO', 1), ('XS', -18), ('AS', 180), ('YT', 'UU')]


We define mapping score as some positive value associated to each alignment that
 measures how good the alignment is. In practice, we use the AS:i secondary flag

1. isfiltrirati neporavnana ocitanja, ona se na kraju ponovo samo nadodaju u reportu (FLAG = 4)

In [5]:
filtered_sam = ""
no_alignment = ""

for read in samfile:
    if read.flag != 4:
        filtered_sam += str(read) + "\n"
    else:
        no_alignment += str(read.query_name) + "NO ALIGNMENT" + "\n"

#print(no_alignment)

In [6]:
samfile.close()

@SQ Reference sequence dictionary. The order of @SQ lines defines the alignment sorting order.    

SN: Reference sequence name. The SN tags and all individual AN names in all @SQ lines must be distinct.
LN: Reference sequence length.   

@PG: Program

EM algoritam za Abundance estimation

In [7]:
import numpy as np
from collections import defaultdict

class AbundanceEstimator:
    def __init__(self, sam_file):
        """
        Initialize the estimator by parsing the SAM file.
        :param sam_file: Path to the SAM file containing alignment information.
        """
        self.references = {}  # {reference_name: reference_length (LN)}
        self.read_assignments = defaultdict(list)  # {read_id: [(reference_name, mapping_score (AS))]}
        self.strain_abundance = {}  # {reference_name: initial abundance estimate}
        self.strain_coverage = {}  # {reference_name: coverage fraction}

        self._parse_sam_file(sam_file)
        self._initialize_abundances()

    def _parse_sam_file(self, sam_file):
        """
        Parse the SAM file to extract reference lengths (from @SQ) and read mappings (AS field).
        """
        samfile = pysam.AlignmentFile(sam_file, "r")

        #header
        for ref_info in samfile.header['SQ']:
            ref_name = ref_info['SN']
            ref_len = ref_info['LN']
            self.references[ref_name] = ref_len

        # Obrada očitanja
        for read in samfile:
            read_id = read.query_name
            ref_name = read.reference_name
            if read.flag != 4:
              if ref_name != None:  # Provjera da očitanje nije nemapirano
                  as_field = read.get_tag('AS') if read.has_tag('AS') else 0
                  if as_field > 0:  # Razmatraj samo mapirana očitanja sa pozitivnim AS poljem
                      if read_id not in self.read_assignments:
                          self.read_assignments[read_id] = []
                      self.read_assignments[read_id].append((ref_name, as_field))

        # Zatvaranje SAM datoteke
        samfile.close()

    def _initialize_abundances(self):
        """
        Compute initial abundance estimates based on mapping scores and reference lengths.
        """
        strain_scores = defaultdict(float)

        for read_id, mappings in self.read_assignments.items():
            total_score = sum(score for _, score in mappings)
            for ref_name, score in mappings:
                ref_len = self.references[ref_name]
                likelihood = score / ref_len  # Normalize by reference length
                strain_scores[ref_name] += likelihood / total_score

        # Normalize abundances to sum to 1
        total_abundance = sum(strain_scores.values())
        for ref_name, score in strain_scores.items():
            self.strain_abundance[ref_name] = score / total_abundance
            self.strain_coverage[ref_name] = strain_scores[ref_name] / total_abundance

    def em_algorithm(self, max_iter=300, eps=1e-3, min_abundance=0.1):
        """
        Run the EM algorithm to refine strain abundances.
        """
        strain_ids = list(self.strain_abundance.keys())
        new_abundance = {strain: 0.0 for strain in strain_ids}
        valid_strains = {strain: True for strain in strain_ids}

        for iteration in range(max_iter):
            # M-korak: Update strain counts using mappings
            for strain in new_abundance:
                new_abundance[strain] = 0.0

            for read_id, mappings in self.read_assignments.items():
                total_prob = 0.0
                probs = []

                for ref_name, score in mappings:
                    if valid_strains[ref_name]:
                        prob = score * self.strain_abundance[ref_name] * self.strain_coverage[ref_name]
                        probs.append((ref_name, prob))
                        total_prob += prob

                if total_prob > 0:
                    for ref_name, prob in probs:
                        new_abundance[ref_name] += prob / total_prob

            # E-korak: Normalize and check for convergence
            total_abundance = sum(new_abundance.values())
            max_diff = 0.0
            converged = True

            for strain in self.strain_abundance:
                if valid_strains[strain]:
                    diff = abs(new_abundance[strain] / total_abundance - self.strain_abundance[strain])
                    max_diff = max(max_diff, diff)
                    if diff > eps:
                        converged = False
                    self.strain_abundance[strain] = new_abundance[strain] / total_abundance

            #print(f"Iteration {iteration + 1}: max_diff = {max_diff}")

            if iteration % 10 == 0:
                self.apply_set_cover(valid_strains, min_abundance)

            if converged:
                break

    def apply_set_cover(self, valid_strains, min_abundance):
        """
        Reduce valid strains using a set cover heuristic.
        """
        removable_strains = {strain for strain, abundance in self.strain_abundance.items()
                             if valid_strains[strain] and abundance < min_abundance}

        # Prevent removing unique strains
        for read_id, mappings in self.read_assignments.items():
            unique_strains = {ref_name for ref_name, _ in mappings if valid_strains[ref_name]}
            if len(unique_strains) == 1:
                removable_strains.discard(next(iter(unique_strains)))

        for strain in removable_strains:
            valid_strains[strain] = False

        #print(f"Set cover reduction applied. Valid strains: {sum(valid_strains.values())}")


no_alignment = ""
sam_file = "/content/drive/My Drive/Colab Notebooks/test.sam"
estimator = AbundanceEstimator(sam_file)
estimator.em_algorithm()
print(dict(estimator.strain_abundance))

{'CHROMOSOME_II': 0.2, 'CHROMOSOME_I': 0.4, 'CHROMOSOME_IV': 0.2, 'CHROMOSOME_V': 0.2}


Re-Assignment of reads

In [8]:
import heapq
import math

class MoraAssignment:
    def __init__(self, references, reads):
        self.references = references  # dictionary referenci i abundance
        self.reads = reads  # dictionary ocitanjna i mogucih poravnanja
        self.assignments = {ref: [] for ref in references}  # odabrana poravnanja ocitanja na reference
        self.reference_capacity = {ref: math.ceil(references[ref] * len(self.reads)) for ref in references} #kapacitet svake reference
        #print(self.reference_capacity)

    def calculate_priority(self, read_id):
        # Reference sortirane od best to worst
        read_mappings = sorted(self.reads[read_id])
        print(read_mappings)
        best_score = read_mappings[0][1]
        if len(read_mappings) == 1: #Postoji samo jedno poravnanje
            return 1
        second_best_score = read_mappings[1][1]
        if second_best_score / best_score < 0.5: # ratio of the second best score to the best score is less than a threshold - inicijalno 0.5 ali staviti da se moze prilagoditi
            return 2
        return 3  # ostalo

    def assign_read(self, read_id, priority):
        read_mappings = self.reads[read_id]
        if priority == 1:
            # Priority 1: ide na jedinu mapiranu referencu
            ref, score = read_mappings[0]
            if self.reference_capacity[ref] > 0:
                self.assignments[ref].append((read_id, score))
                self.reference_capacity[ref] -= 1
            return True
        elif priority == 2:
            # Priority 2: reads are sorted and then assigned to the reference with the best mapping score if that reference has space. If the reference is at full capacity, the read is relabeled as a priority 3 read
            read_mappings.sort(key=lambda x: -x[1])
            for ref, score in read_mappings:
                if self.reference_capacity[ref] > 0:
                    self.assignments[ref].append((read_id, score))
                    self.reference_capacity[ref] -= 1
                    return True
            return False
        elif priority == 3:
            # Priority 3: assigned to best possible reference or left over for a second round of assignment if all of its potential references are full
            read_mappings.sort(key=lambda x: -x[1])  # Sort by score (descending)
            for ref, score in read_mappings:
                if self.reference_capacity[ref] > 0:
                    self.assignments[ref].append((read_id, score))
                    self.reference_capacity[ref] -= 1
                    return True
            return False

    def reassign_read(self, read_id):
        # Mora will try to “open up space” in a reference to assign leftover reads.
        read_mappings = self.reads[read_id]
        for ref, score in read_mappings:
            if len(self.assignments[ref]) > 0:
                for assigned_read, assigned_score in self.assignments[ref]:
                    if assigned_score < score:  # Mice read sa manjim AS
                        for new_ref, new_score in self.reads[assigned_read]:
                            if self.reference_capacity[new_ref] > 0:
                                self.assignments[new_ref].append((assigned_read, assigned_score))
                                self.assignments[ref].remove((assigned_read, assigned_score))
                                self.reference_capacity[ref] += 1
                                print(self.reference_capacity)
                                self.reference_capacity[new_ref] -= 1
                                print(self.reference_capacity)
                                self.assignments[ref].append((read_id, score))
                                return True
        return False

    def assign_reads(self):
        # 1. odredi prioritet
        for read_id in self.reads:
            priority = self.calculate_priority(read_id)
            if not self.assign_read(read_id, priority):
                # Preraspodjela za kada nema kapaciteta
                self.reassign_read(read_id)


mora = MoraAssignment(estimator.strain_abundance, dict(estimator.read_assignments))
mora.assign_reads()

print(mora.assignments)


[('CHROMOSOME_II', 180)]
[('CHROMOSOME_I', 180)]
[('CHROMOSOME_IV', 180)]
[('CHROMOSOME_I', 180)]
[('CHROMOSOME_V', 180)]
{'CHROMOSOME_II': [('II.14978392', 180)], 'CHROMOSOME_I': [('III', 180), ('V', 180)], 'CHROMOSOME_IV': [('IV', 180)], 'CHROMOSOME_V': [('VI', 180)]}


In [9]:
result = ""
for reference, reads in mora.assignments.items():
    for read, score in reads:
        result += f"{read}\t{reference}\n"

result += no_alignment
print(result)

II.14978392	CHROMOSOME_II
III	CHROMOSOME_I
V	CHROMOSOME_I
IV	CHROMOSOME_IV
VI	CHROMOSOME_V

